# On Jean Zay HPC

```bash
module purge

module load arch/h100
module load cuda/12.4.1
module load ffmpeg/6.1.1

module load miniforge

conda activate jsalt10
```

If you want to perform dScaper and PyRoomAcoustics (steps 2 and 3):
```bash
cd ./dscaper
pip install -e .

conda install sox
pip install sox

pip install jams
pip install pyloudnorm
```

# SDialog dependencies

In [33]:
%%script false --no-raise-error
# Only run this cell if you are using a non jupyter environment
!conda create --name jsalt python=3.9 -y
!conda activate jsalt
!conda install sox
!pip install -r sdialog/requirements.txt
!pip install -r sdialog/requirements-audio.txt
#  conda activate jsalt-Py3-10

In [34]:
import os
import json
from tqdm import tqdm

In [35]:
import sdialog
from sdialog import Dialog
from sdialog.generators import PersonaGenerator
from sdialog.personas import Persona, PersonaAgent, Doctor, Patient, Agent

# Generate Persona

In [36]:
sdialog.config.set_llm("aws:anthropic.claude-3-5-sonnet-20240620-v1:0", region_name="us-east-1")

In [37]:
%%script false --no-raise-error
persona_card_folder = "./personas"

# Generate doctor persona
doctor_persona = Doctor(
    name="Dr. Smith",
    gender="male",
    age=52,
    specialty="Family Medicine"
)
generator_doctor = PersonaGenerator(doctor_persona)
persona_cards = generator_doctor.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_doctor.json")

# Generate patient persona
patient_persona = Patient(
    name="John Doe",
    gender="male",
    age=62
)
generator_patient = PersonaGenerator(patient_persona)
persona_cards = generator_patient.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_patient.json")


# Load persona

In [38]:
# Load personas
persona_doctor = Persona.from_file("./personas/persona_doctor.json")
persona_patient = Persona.from_file("./personas/persona_patient.json")

In [39]:
%%script false --no-raise-error
context = "Generate me a 50 turn medical dialogue between patient and doctor, for a primary care visit"

# Create agents
agent1 = PersonaAgent(persona=persona_doctor, name="DOCTOR", dialogue_details=context, response_details="make short turn answers when needed")
agent2 = PersonaAgent(persona=persona_patient, name="PATIENT", dialogue_details=context, response_details="make short turn answers when needed")

In [40]:
save_all = True
GENERATE_PERSONA = True
FORCE_DIALOG_GENERATION = False

os.makedirs("./outputs", exist_ok=True)

In [41]:
if FORCE_DIALOG_GENERATION:

    original_dialog = agent1.talk_with(agent2, max_turns=3)
    original_dialog.to_file("dialog_demo.json")

else:
    original_dialog = Dialog.from_file("dialog_demo.json")

original_dialog.print()

[dialog_id] 1752861530588
[model] client=<botocore.client.BedrockRuntime object at 0x7fc319ec91c0> model_id='anthropic.claude-3-5-sonnet-20240620-v1:0' region_name='us-east-1' provider='anthropic' supports_tool_choice_values=('auto', 'any', 'tool')
[seed] 226296126
--- Dialogue Begins ---
[DOCTOR] Hello there. I'm Dr. Smith. Welcome to my office. What brings you in to see me today?
[PATIENT] Good morning, Dr. Smith. I'm John Doe. I've been dealing with a persistent cough and feeling pretty tired for about three weeks now. It's starting to wear me down, and I thought I should get it checked out.
[DOCTOR] I'm sorry to hear you've been feeling unwell, Mr. Doe. A persistent cough and fatigue can certainly be troublesome. Let's get some more details. Can you describe your cough? Is it dry or productive?
[PATIENT] Well, Dr. Smith, it's mostly a dry cough. It's particularly bothersome at night when I'm trying to sleep. You know, it reminds me of the time I was teaching about the Civil War and

# Audio Generation

You can generate three type of audios:
- (default) Step 1: Raw utterances passed to a TTS model and concatenated to each others to create an audio file
- Step 2: Audio generated from multiple channels create using signal positions
- Step 3: Audio generated using room spacialization and multi-channels positions

If you want to trigger the "step 2" you need to give a Scaper argument to the `audio_pipeline`. While for the "step 3" you need also to give a "Room" in the `inference` function.

### Instanciate voices database

In [42]:
from sdialog.audio.voice_database import DummyKokoroVoiceDatabase
dummy_voice_database = DummyKokoroVoiceDatabase()
dummy_voice_database.get_voice(genre="male", age=20)

{'identifier': 'am_adam', 'voice': 'am_adam'}

OR

In [43]:
%%script false --no-raise-error
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
voices_libritts = HuggingfaceVoiceDatabase("sdialog/voices-libritts")
voices_libritts.get_voice(genre="male", age=20)

OR

In [44]:
%%script false --no-raise-error
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
dummy_voice_database = HuggingfaceVoiceDatabase("sdialog/voices-jsalt")
dummy_voice_database.get_voice(genre="male", age=20)

### Instanciate TTS model

In [45]:
from sdialog.audio.tts_engine import KokoroTTS
tts_pipeline = KokoroTTS()

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


OR

In [46]:
%%script false --no-raise-error
from sdialog.audio.tts_engine import IndexTTS
tts_pipeline = IndexTTS(device="cpu")

In [47]:
%%script false --no-raise-error
import soundfile
audio, sampling_rate = tts_pipeline.generate(
    "Brno is the best city in the planet, you know? and Loco Polaco is the craziest person I know",
    "./sergio.wav"
)
soundfile.write("./test_index_tts.wav", audio, sampling_rate)

## Setup stage: Audio Dialog and Audio Pipeline

In [48]:
from sdialog.audio.audio_dialog import AudioDialog
from sdialog.audio.audio_pipeline import AudioPipeline

In [49]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)

## Step 1 : Concatenated utterances

In [50]:
audio_pipeline = AudioPipeline(
    voice_database=dummy_voice_database,
    tts_pipeline=tts_pipeline,
    dir_audio="./outputs",
)

OR

In [51]:
%%script false --no-raise-error
audio_pipeline = AudioPipeline() # Default values are used

In [52]:
dialog: AudioDialog = audio_pipeline.inference(dialog) # Generate the audio for the dialog
print(dialog.audio_step_1_filepath) # Path to the audio of the first stage of the audio pipeline

./outputs/dialog_1752861530588/exported_audios/audio_pipeline_step1.wav
Generating utterances audios from dialogue 1752861530588


Generating utterances audios: 100%|██████████| 4/4 [00:39<00:00,  9.95s/it]
[2025-08-18 21:58:07] WARNING:root:The dSCAPER is not set, which make the generation of the timeline impossible
[2025-08-18 21:58:07] WARNING:root:The room or the dSCAPER is not set, which make the generation of the room accoustic audio impossible


./outputs/dialog_1752861530588/exported_audios/audio_pipeline_step1.wav


## Step 2: dScaper

In [53]:
%%script false --no-raise-error
!git clone https://github.com/cyrta/dscaper.git

In [54]:
%%script false --no-raise-error
%pip install -r ../../../requirements-dscaper.txt

In [55]:
import scaper
DATA_PATH = "./dscaper_data" # Path where the sound events, utterances and timelines database will be saved
os.makedirs(DATA_PATH, exist_ok=True)

In [56]:
dsc = scaper.Dscaper(dscaper_base_path=DATA_PATH)

In [57]:
audio_pipeline = AudioPipeline(dscaper=dsc)

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [58]:
# Populate the sound events database
audio_pipeline.populate_dscaper(["sdialog/background","sdialog/foreground"])

Populating dSCAPER with sdialog/background dataset...:   0%|          | 0/4 [00:00<?, ?it/s][2025-08-18 21:58:16] ERROR:root:Problem storing audio /Users/yanislabrak/.cache/huggingface/hub/datasets--sdialog--background/snapshots/899f03337bf907855fa99ececdadbcabf3705ab6/ac_noise/351790__reiyamanor__small-room-tone-vintage-air-condition-at.wav (the audio can also be already stored)
Populating dSCAPER with sdialog/background dataset...:  25%|██▌       | 1/4 [00:00<00:01,  2.01it/s][2025-08-18 21:58:16] ERROR:root:Problem storing audio /Users/yanislabrak/.cache/huggingface/hub/datasets--sdialog--background/snapshots/899f03337bf907855fa99ececdadbcabf3705ab6/ac_noise/546572__tim_verberne__f_st_room_tone_04.wav (the audio can also be already stored)
[2025-08-18 21:58:16] ERROR:root:Problem storing audio /Users/yanislabrak/.cache/huggingface/hub/datasets--sdialog--background/snapshots/899f03337bf907855fa99ececdadbcabf3705ab6/fan_noise/210098__yuval__room-big-fan.wav (the audio can also be alre

0

In [59]:
dialog: AudioDialog = audio_pipeline.inference(dialog)
print(dialog.audio_step_1_filepath)
print(dialog.audio_step_2_filepath)

[2025-08-18 21:58:19] ERROR:root:Problem storing audio for turn ./outputs/dialog_1752861530588/utterances/0_DOCTOR.wav
[2025-08-18 21:58:19] ERROR:root:Problem storing audio for turn ./outputs/dialog_1752861530588/utterances/1_PATIENT.wav
[2025-08-18 21:58:19] ERROR:root:Problem storing audio for turn ./outputs/dialog_1752861530588/utterances/2_DOCTOR.wav
[2025-08-18 21:58:19] ERROR:root:Problem storing audio for turn ./outputs/dialog_1752861530588/utterances/3_PATIENT.wav
/Users/yanislabrak/Desktop/HUB/JSALT/sdialog/dscaper/scaper/core.py:1657: ScaperWarning: DOCTOR event duration (5.50) is greater that source duration (5.15), changing to 5.15
  warnings.warn(
/Users/yanislabrak/Desktop/HUB/JSALT/sdialog/dscaper/scaper/core.py:1657: ScaperWarning: DOCTOR event duration (12.10) is greater that source duration (11.05), changing to 11.05
  warnings.warn(
/Users/yanislabrak/Desktop/HUB/JSALT/sdialog/dscaper/scaper/core.py:1657: ScaperWarning: PATIENT event duration (10.80) is greater that

./outputs/dialog_1752861530588/exported_audios/audio_pipeline_step1.wav
Loading utterances and combined audio from dialogue 1752861530588
Audio data from step 1 loaded into the dialog (1752861530588) successfully!
Sending utterances to dSCAPER for dialogue 1752861530588
Generating timeline from dSCAPER for dialogue 1752861530588


/Users/yanislabrak/Desktop/HUB/JSALT/sdialog/dscaper/scaper/core.py:2286: ScaperWarning: Soundscape audio is clipping!
  warnings.warn('Soundscape audio is clipping!',
/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/soxbindings/effects.py:93: RuntimeWarning: invalid value encountered in cast
  input_data = input_data.astype(np.int32)
[2025-08-18 21:58:21] INFO:root:Successfully generated dscaper timeline.
[2025-08-18 21:58:21] WARNING:root:The room or the dSCAPER is not set, which make the generation of the room accoustic audio impossible


Timeline generated from dSCAPER for dialogue 1752861530588
./outputs/dialog_1752861530588/exported_audios/audio_pipeline_step1.wav
./outputs/dialog_1752861530588/exported_audios/audio_pipeline_step2.wav


## Step 3 : Room Accoustics

In [60]:
audio_pipeline = AudioPipeline(dscaper=dsc) # The audio pipeline doesn't change

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [61]:
from sdialog.audio.room import MicrophonePosition
from sdialog.audio.room_generator import RoomGenerator, RoomRole

In [62]:
room = RoomGenerator().generate(RoomRole.CONSULTATION, room_size=8.0)
print(room)

7107:  RoomRole.CONSULTATION room_7107, desc: consultation room (dimentions: dim: [3.577708763999664, 2.23606797749979, 3.0], rt60: 0.5) role: RoomRole.CONSULTATION)  


In [64]:
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    room=room, # Need to provide a room object to trigger the 3rd step of the audio pipeline
    # microphone_position=MicrophonePosition.MONITOR # Default is MicrophonePosition.MONITOR
    microphone_position=MicrophonePosition.CEILING_CENTERED, # Default is MicrophonePosition.MONITOR
    do_step_1=True,
    do_step_2=False,
    do_step_3=True,
)
print(dialog.audio_step_1_filepath)
print(dialog.audio_step_2_filepath)
print(dialog.audio_step_3_filepath)

./outputs/dialog_1752861530588/exported_audios/audio_pipeline_step1.wav
Loading utterances and combined audio from dialogue 1752861530588
Audio data from step 1 loaded into the dialog (1752861530588) successfully!
Generating room accoustic for dialogue 1752861530588
  Microphone set to position [0.9944271909999159, 0.33541019662496846, 1.2]
  Microphone set to position [1.788854381999832, 1.118033988749895, 2.9]
✓ Loaded audio file './dscaper_data/timelines/dialog_1752861530588/generate/f3cd3044-1c11-4016-a47c-b24bc2a664ea/soundscape_positions/no_type.wav' for 'no_type' with 1995525 samples
✓ Loaded audio file './dscaper_data/timelines/dialog_1752861530588/generate/f3cd3044-1c11-4016-a47c-b24bc2a664ea/soundscape_positions/doctor-at_desk_sitting.wav' for 'doctor-at_desk_sitting' with 1995525 samples
✓ Loaded audio file './dscaper_data/timelines/dialog_1752861530588/generate/f3cd3044-1c11-4016-a47c-b24bc2a664ea/soundscape_positions/patient-next_to_desk_sitting.wav' for 'patient-next_to_d